In [13]:
# import glob
# import os
# import argparse
# import json
# import torch
# import librosa
# from models.stfts import mag_phase_stft, mag_phase_istft
# from models.generator import SEMamba
# from models.pcs400 import cal_pcs
# import soundfile as sf

# from utils.util import (
#     load_ckpts, load_optimizer_states, save_checkpoint,
#     build_env, load_config, initialize_seed, 
#     print_gpu_info, log_model_info, initialize_process_group,
# )

# import numpy as np

In [1]:
from models.generator import SEMamba
import torch
from utils.util import load_config

In [2]:
print('Initializing Inference Process..')
# 修改 config 為有效的 YAML 配置文件路徑
config = '/disk4/chocho/SEMamba/exp/20250403-SEMamba_v1_PCS-h32-tf2/config.yaml'  # 替換為實際的 YAML 檔案路徑
checkpoint_file = '/disk4/chocho/SEMamba/exp/20250403-SEMamba_v1_PCS-h32-tf2/g_00078000.pth'

global device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    #device = torch.device('cpu')
    raise RuntimeError("Currently, CPU mode is not supported.")

cfg = load_config(config)
model = SEMamba(cfg).to(device)
state_dict = torch.load(checkpoint_file, map_location=device)
model.load_state_dict(state_dict['generator'])

Initializing Inference Process..


<All keys matched successfully>

In [3]:
# from mamba_ssm.modules.mamba_simple import Mamba
import torch
# from mamba_ssm import Mamba
from torchinfo import summary
# from torchprofile import profile_macs

batch, length, dim = 5, 201, 286
print("B,L,D:",batch, length, dim)
x = torch.randn(batch, length, dim).to("cuda")

# summary_str = summary(model, input_size=[(7, 201, 286), (7, 201, 286)], depth=5, col_names=("input_size", "output_size", "num_params"), verbose=0)
summary_str = summary(model, input_size=[x.shape, x.shape], depth=15, col_names=("input_size", "output_size", "num_params"), verbose=0)
print(summary_str)

# y = model(x)
# print(x.shape)
# print(y.shape)
# assert y.shape == x.shape

B,L,D: 5 201 286
Layer (type:depth-idx)                             Input Shape               Output Shape              Param #
SEMamba                                            [5, 201, 286]             [5, 201, 286]             --
├─DenseEncoder: 1-1                                [5, 2, 286, 201]          [5, 32, 286, 100]         --
│    └─Sequential: 2-1                             [5, 2, 286, 201]          [5, 32, 286, 201]         --
│    │    └─Conv2d: 3-1                            [5, 2, 286, 201]          [5, 32, 286, 201]         96
│    │    └─InstanceNorm2d: 3-2                    [5, 32, 286, 201]         [5, 32, 286, 201]         64
│    │    └─PReLU: 3-3                             [5, 32, 286, 201]         [5, 32, 286, 201]         32
│    └─DenseBlock: 2-2                             [5, 32, 286, 201]         [5, 32, 286, 201]         --
│    │    └─ModuleList: 3-4                        --                        --                        --
│    │    │    └─Sequent

In [4]:
from calflops import calculate_flops

batch, length, dim = 5, 201, 286
print("B,L,D:",batch, length, dim)
noisy_amp = torch.randn(batch, length, dim).to("cuda")
noisy_pha = torch.randn(batch, length, dim).to("cuda")

with torch.no_grad():
    # 使用 calflops 計算 FLOPs，將 args 改為列表
    flops, macs, params = calculate_flops(
        model=model,
        args=[noisy_amp, noisy_pha],  # 使用列表而非元組
        print_results=True  # 顯示逐層結果
    )
    # print(f"Total FLOPs for {fname}: {flops / 1e9:.3f} GFLOPs")
    # print(f"Total Params: {params / 1e6:.3f} M")
    # print(f"Total MACs: {macs / 1e9:.3f} GMACs")
    print(f"Total FLOPs: {flops}")
    print(f"Total Params: {params}")
    print(f"Total MACs: {macs}")

B,L,D: 5 201 286

------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  455.31 K
fwd MACs:                                                               65.05 GMACs
fwd FLOPs:                                                              130.79 GFLOPS
fwd+bwd MACs:                                                           195.14 GMACs
fwd+bwd FLOPs:                                                          392.36 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results ----------------------------

In [5]:
from calflops import calculate_flops

shape = [5,2,286,201]
with torch.no_grad():
    # 使用 calflops 計算 FLOPs，將 args 改為列表
    flops, macs, params = calculate_flops(
        model=model.dense_encoder,
        args=[torch.randn(shape).to("cuda")],  # 使用列表而非元組
        print_results=True  # 顯示逐層結果
    )
    # print(f"Total FLOPs for {fname}: {flops / 1e9:.3f} GFLOPs")
    # print(f"Total Params: {params / 1e6:.3f} M")
    # print(f"Total MACs: {macs / 1e9:.3f} GMACs")
    print(f"Total FLOPs: {flops}")
    print(f"Total Params: {params}")
    print(f"Total MACs: {macs}")


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  96.06 K 
fwd MACs:                                                               26.95 GMACs
fwd FLOPs:                                                              54.25 GFLOPS
fwd+bwd MACs:                                                           80.84 GMACs
fwd+bwd FLOPs:                                                          162.75 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each module ca

In [6]:
model.dense_encoder.dense_conv_1

Sequential(
  (0): Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1))
  (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (2): PReLU(num_parameters=32)
)

In [7]:
model.dense_encoder.dense_conv_1[0].weight.shape, \
    model.dense_encoder.dense_conv_1[0].bias.shape

(torch.Size([32, 2, 1, 1]), torch.Size([32]))

In [8]:
model.dense_encoder.dense_conv_1[1].weight.shape, \
    model.dense_encoder.dense_conv_1[1].bias.shape

(torch.Size([32]), torch.Size([32]))

In [9]:
model.dense_encoder.dense_conv_1[2].weight.shape

torch.Size([32])

In [10]:
with torch.no_grad():
    model.dense_encoder.dense_conv_1[1].weight.data = torch.ones_like(model.dense_encoder.dense_conv_1[1].weight)
model.dense_encoder.dense_conv_1[1].weight    

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', requires_grad=True)

In [11]:
model.dense_encoder.dense_conv_1[1].weight,

(Parameter containing:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        device='cuda:0', requires_grad=True),)

In [12]:
model.dense_encoder.dense_conv_1[2].weight,

(Parameter containing:
 tensor([-0.0352, -0.0052, -0.0340, -0.0106,  0.1124,  0.2487,  0.1829,  0.0049,
         -0.0568,  0.2071, -0.0741,  0.1177,  0.1433, -0.0592,  0.0790, -0.0420,
          0.0500,  0.3584,  0.0108,  0.1113, -0.0980,  0.0066,  0.1041, -0.0105,
          0.1420,  0.3210,  0.0218, -0.0035, -0.0601, -0.2030,  0.1472, -0.0635],
        device='cuda:0', requires_grad=True),)

In [14]:
model.TSMamba[0].time_mamba.forward_blocks[0]

Block(
  (mixer): Mamba(
    (in_proj): Linear(in_features=32, out_features=256, bias=False)
    (conv1d): Conv1d(128, 128, kernel_size=(4,), stride=(1,), padding=(3,), groups=128)
    (act): SiLU()
    (x_proj): Linear(in_features=128, out_features=34, bias=False)
    (dt_proj): Linear(in_features=2, out_features=128, bias=True)
    (out_proj): Linear(in_features=128, out_features=32, bias=False)
  )
  (norm): RMSNorm()
)

In [15]:
model.TSMamba[0].time_mamba.forward_blocks[0].norm

RMSNorm()

In [14]:
model.TSMamba[0].time_mamba.forward_blocks[0].mixer

Mamba(
  (in_proj): Linear(in_features=32, out_features=256, bias=False)
  (conv1d): Conv1d(128, 128, kernel_size=(4,), stride=(1,), padding=(3,), groups=128)
  (act): SiLU()
  (x_proj): Linear(in_features=128, out_features=34, bias=False)
  (dt_proj): Linear(in_features=2, out_features=128, bias=True)
  (out_proj): Linear(in_features=128, out_features=32, bias=False)
)

In [16]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        # 這裡假設 base_model 是一個有 TSMamba[0].time_mamba.forward_blocks[0] 結構的模型
        self.first_block = base_model.dense_encoder.dense_conv_1[0]

    def forward(self, x):
        x = self.first_block(x)
        return x
# 假設你已經有一個已初始化好的 base_model
my_model = CustomModel(base_model=model)

# 做 forward 傳入輸入資料
shape = [1,2,286,201]
input_tensor = torch.randn(shape).to("cuda")
output = my_model(input_tensor)
output, \
    output.shape

(tensor([[[[ 7.1235e-03, -1.0601e-02,  2.9499e-01,  ...,  1.1048e-01,
             1.1252e-01, -2.2514e-01],
           [ 2.2336e-01,  2.5662e-01,  1.0795e-01,  ...,  1.7424e-01,
             2.3170e-01, -1.1843e-01],
           [ 1.0530e-01,  4.6253e-02,  2.5955e-01,  ..., -6.2019e-02,
             1.2044e-01,  1.5700e-01],
           ...,
           [ 2.8144e-02,  2.0385e-01, -1.2203e-01,  ...,  7.4711e-03,
             2.4381e-02,  1.6448e-01],
           [-1.3493e-01,  7.5547e-02, -5.6866e-03,  ..., -1.9580e-02,
            -5.9942e-02,  2.5650e-02],
           [ 7.2733e-02,  2.4791e-01,  2.1365e-01,  ..., -2.9166e-01,
            -1.7886e-01,  5.6788e-02]],
 
          [[-3.9917e-01, -5.5537e-01,  3.9121e-01,  ...,  6.4231e-03,
            -7.2457e-02, -1.2468e+00],
           [ 3.0526e-01,  3.6878e-01, -1.3122e-01,  ...,  2.0159e-01,
             9.3846e-02, -1.0260e+00],
           [-1.4060e-01, -3.7322e-01,  3.4110e-01,  ..., -7.1094e-01,
            -1.1940e-01,  8.2347e-02],


In [17]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        # 這裡假設 base_model 是一個有 TSMamba[0].time_mamba.forward_blocks[0] 結構的模型
        self.first_block = base_model.TSMamba[0].time_mamba.forward_blocks[0].mixer

    def forward(self, x):
        x = self.first_block(x)
        return x
# 假設你已經有一個已初始化好的 base_model
my_model = CustomModel(base_model=model)

# 做 forward 傳入輸入資料
shape = [100,286,32]
input_tensor = torch.randn(shape).to("cuda")
output = my_model(input_tensor)
output, \
    output.shape

(tensor([[[ 0.0847, -0.0665, -0.0233,  ...,  0.0078, -0.1086,  0.0164],
          [-0.0798,  0.0263, -0.0892,  ...,  0.1521,  0.0422, -0.2575],
          [ 0.0147, -0.0233,  0.0099,  ...,  0.1294,  0.0259, -0.0891],
          ...,
          [-0.0520,  0.1250,  0.3329,  ..., -0.1316,  0.0767,  0.1543],
          [ 0.0534,  0.5691,  0.3958,  ..., -0.3923, -0.3511,  0.0061],
          [-0.0040,  0.1272,  0.1016,  ..., -0.3203, -0.0197, -0.3803]],
 
         [[-0.0299,  0.0807,  0.0739,  ..., -0.1404,  0.0129, -0.2082],
          [ 0.1873, -0.0825,  0.1137,  ...,  0.0411,  0.1657,  0.0345],
          [ 0.3254,  0.0703, -0.2580,  ..., -0.0165, -0.0960,  0.0337],
          ...,
          [ 0.4094,  0.4627, -0.2452,  ...,  0.1872,  0.1634,  0.0524],
          [ 0.2161, -0.0326,  0.1320,  ...,  0.0499, -0.1213, -0.1543],
          [ 0.0373, -0.1468, -0.0747,  ...,  0.0610, -0.4242, -0.1850]],
 
         [[-0.0614, -0.0033, -0.0147,  ..., -0.0262, -0.0140, -0.0195],
          [-0.0478,  0.0734,